In [3]:
from sqlalchemy import Column
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship

Base = declarative_base()

class User(Base):
    __tablename__ = "user_account"
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship(
        "Address", back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)
    user = relationship("User", back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [4]:
from sqlalchemy import create_engine
engine = create_engine("sqlite://", echo=True, future=True)

In [6]:
Base.metadata.create_all(engine)

2022-05-15 19:33:08,291 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-15 19:33:08,293 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-05-15 19:33:08,293 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 19:33:08,294 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-05-15 19:33:08,295 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 19:33:08,296 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()


2022-05-15 19:33:23,309 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-15 19:33:23,311 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-05-15 19:33:23,312 INFO sqlalchemy.engine.Engine [generated in 0.00112s] ('spongebob', 'Spongebob Squarepants')
2022-05-15 19:33:23,313 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-05-15 19:33:23,314 INFO sqlalchemy.engine.Engine [cached since 0.002834s ago] ('sandy', 'Sandy Cheeks')
2022-05-15 19:33:23,315 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-05-15 19:33:23,316 INFO sqlalchemy.engine.Engine [cached since 0.004454s ago] ('patrick', 'Patrick Star')
2022-05-15 19:33:23,318 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-05-15 19:33:23,318 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ('spongebob@sqlalchemy.org', 1)
2022-05-15 19:33:23,319 INFO sqlalchemy

In [8]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)


2022-05-15 19:34:05,853 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-15 19:34:05,857 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2022-05-15 19:34:05,858 INFO sqlalchemy.engine.Engine [generated in 0.00121s] ('spongebob', 'sandy')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')


In [9]:
stmt = (
 select(Address)
 .join(Address.user)
 .where(User.name == "sandy")
 .where(Address.email_address == "sandy@sqlalchemy.org")
)
sandy_address = session.scalars(stmt).one()


sandy_address

2022-05-15 19:34:34,645 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2022-05-15 19:34:34,646 INFO sqlalchemy.engine.Engine [generated in 0.00133s] ('sandy', 'sandy@sqlalchemy.org')


Address(id=2, email_address='sandy@sqlalchemy.org')

In [10]:
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()


patrick.addresses.append(
    Address(email_address="patrickstar@sqlalchemy.org")
)


sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"

session.commit()


2022-05-15 19:35:59,663 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-05-15 19:35:59,664 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ('patrick',)
2022-05-15 19:35:59,667 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-05-15 19:35:59,667 INFO sqlalchemy.engine.Engine [generated in 0.00093s] (3,)
2022-05-15 19:35:59,670 INFO sqlalchemy.engine.Engine UPDATE address SET email_address=? WHERE address.id = ?
2022-05-15 19:35:59,671 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ('sandy_cheeks@sqlalchemy.org', 2)
2022-05-15 19:35:59,672 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-05-15 19:35:59,672 INFO sqlalchemy.engine.Engine [cached since 156.4s ago] ('patrickstar@sqlalchemy.org', 3

In [11]:
sandy = session.get(User, 2)


sandy.addresses.remove(sandy_address)


2022-05-15 19:36:46,936 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-15 19:36:46,938 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-05-15 19:36:46,939 INFO sqlalchemy.engine.Engine [generated in 0.00085s] (2,)
2022-05-15 19:36:46,941 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-05-15 19:36:46,941 INFO sqlalchemy.engine.Engine [cached since 47.27s ago] (2,)


In [12]:
session.flush()


2022-05-15 19:36:51,227 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2022-05-15 19:36:51,228 INFO sqlalchemy.engine.Engine [generated in 0.00121s] (2,)


In [13]:
session.delete(patrick)


2022-05-15 19:36:56,342 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-05-15 19:36:56,343 INFO sqlalchemy.engine.Engine [cached since 9.405s ago] (3,)
2022-05-15 19:36:56,345 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-05-15 19:36:56,345 INFO sqlalchemy.engine.Engine [cached since 56.68s ago] (3,)


In [14]:
session.commit()


2022-05-15 19:37:04,195 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2022-05-15 19:37:04,196 INFO sqlalchemy.engine.Engine [cached since 12.97s ago] (4,)
2022-05-15 19:37:04,197 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2022-05-15 19:37:04,198 INFO sqlalchemy.engine.Engine [generated in 0.00088s] (3,)
2022-05-15 19:37:04,199 INFO sqlalchemy.engine.Engine COMMIT
